# <center> TLS Deep Dive
## <center> SYSE 549: Secure Vehicle and Industrial Networking
## <center> <img src="https://www.engr.colostate.edu/~jdaily/Systems-EN-CSU-1-C357.svg" width="400" /> 
### <center> Instructor: Dr. Jeremy Daily<br>
### <center> Content Author: Jerry Duggan

## Learning Objectives
By the end of this module, you will:
* Understand how to create and verify a self-signed certificate;
* Understand how to initiate a TLS v1.2 session with an HTTP server;
* Understand how to access components of a TLS certificate with a client application, and how to verify the certificate;
* Understand how to use Wireshark to examine the steps for establishing a TLS session.

## Quick Review

![Person in the Middle](PITM.png)

![Certificates](CA.png)

## Create & verify a self-signed certificate

### Load dependencies

In [31]:
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import hashes
from cryptography.exceptions import InvalidSignature
from datetime import datetime, timedelta
from cryptography.hazmat.backends import default_backend
from cryptography import x509
from cryptography.hazmat.primitives.asymmetric import padding

### Generate private key & calculate it's associated public key

In [10]:
privateKey = rsa.generate_private_key(
        public_exponent=65537,
        key_size=2048,
        backend=default_backend()
    )

publicKey = privateKey.public_key()
print(publicKey)

### Create an x509 object with our pertinent information

In [11]:
issuer = x509.Name([
        x509.NameAttribute(x509.NameOID.COUNTRY_NAME,           u"US"),
        x509.NameAttribute(x509.NameOID.STATE_OR_PROVINCE_NAME, u"Colorado"),
        x509.NameAttribute(x509.NameOID.LOCALITY_NAME,          u"Fort Collins"),
        x509.NameAttribute(x509.NameOID.ORGANIZATION_NAME,      u"Colorado State University"),
        x509.NameAttribute(x509.NameOID.COMMON_NAME,            u"fakesite.edu")
    ])
issuer

<Name(C=US,ST=Colorado,L=Fort Collins,O=Colorado State University,CN=fakesite.edu)>

### This is a self-signed certificate, so in the case the issuer is also the subject

In [12]:
subject = issuer

### Create the certificate builder...

In [13]:
rawCert = (
        x509.CertificateBuilder()
            .subject_name(subject)
            .issuer_name(issuer)
            .public_key(publicKey)
            .serial_number(x509.random_serial_number())
            .not_valid_before(datetime.utcnow())
            .not_valid_after(datetime.utcnow() + timedelta(days=10))
            .add_extension(x509.SubjectAlternativeName([x509.DNSName(u"localhost")]), critical=False)
            )
rawCert

### And sign it with our private key to make it a real certificate

In [14]:
signedCert = rawCert.sign(privateKey, hashes.SHA256(), backend=default_backend())
signedCert

<Certificate(subject=<Name(C=US,ST=Colorado,L=Fort Collins,O=Colorado State University,CN=fakesite.edu)>, ...)>

### Pretend the public key & certificate were transmitted...

### And now the "client" verifies it

In [15]:
try:
    publicKey.verify(
            signedCert.signature,
            signedCert.tbs_certificate_bytes,
            padding.PKCS1v15(),
            signedCert.signature_hash_algorithm
        )
    print("Valid signature")
except InvalidSignature as e:
    print("Invalid signature")

Valid signature


## TLS Deep Dive

To prepare for this section:
* Open Wireshark
* Set a capture filter to 'host 140.82.112.3 and port 443'
* Set a display filter to 'tls'

### Establish a Socket Connection

We have seen how to establish a socket connection before.  The folks who implement TLS in Python have gone to a lot of trouble to make it transpartent to use a TLS-enabled socket.  The problem is that the normal ways of doing so "hide" a lot of the details, and we explicitly want access to those details.  So we have to go to a bit of extra trouble to get this to work.

### Load Prereqs

In [26]:
%pip install pyOpenSSL


[notice] A new release of pip available: 22.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [27]:
from OpenSSL import SSL
import certifi
import socket
from cryptography import x509
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.asymmetric import padding

AttributeError: module 'lib' has no attribute 'X509_V_FLAG_CB_ISSUER_CHECK'

### Parameters to Use

In [28]:
HOSTNAME = 'api.github.com'
DEST_IPADDR = '140.82.112.3'
PORT = 443

### Set up an SSL context, which will allow us to control and query a lot of the parameters for the exchange

In [30]:

context = OpenSSL.SSL.Context(method=SSL.TLSv1_2_METHOD)
context.load_verify_locations(cafile=certifi.where())

AttributeError: module 'lib' has no attribute 'X509_V_FLAG_CB_ISSUER_CHECK'

### Create the socket connection

The call to 'do_handshake' does all the work required establish the connection.

Bring up your Wireshark window now

In [18]:
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    conn = SSL.Connection(context, socket=s)
    conn.settimeout(5)
    conn.connect((DEST_IPADDR, PORT))
    conn.setblocking(1)
    conn.do_handshake()
    peerCert = conn.get_peer_certificate()  # Get the certificate from the packet
    certChain = conn.get_peer_cert_chain()  # Get the certificates of interest from the local trust store


### Extract the certs we care about 

TBH, we are using two different cryptography libraries here.  I'm using OpenSSL only because it has a call to extract the certificate chain from the message.  I'd rather use the cryptography library (which we've looked at before) to do the heavy lifting...

In [17]:
CACert = certChain[-1].to_cryptography()
pktCert = peerCert.to_cryptography()

### Repeat the steps from the sequence diagram...

![Certificates](CA.png)

#### Verify the certificate with the public key from our local cert store

In [40]:
#
# Verify certificate w/ CA public key
#

try:
    CACert.public_key().verify(
            pktCert.signature,
            pktCert.tbs_certificate_bytes,
            padding.PKCS1v15(),
            pktCert.signature_hash_algorithm
        )
    print("Valid signature")
except InvalidSignature as e:
    print("Invalid signature")

Invalid signature


#### Verify the hostname & certificate validity times

In [41]:
# todo

#### Verify the hashes

In [ ]:
# todo